### Loading Raw(ish) Data 

#### Define table schemas 

In [146]:
import os
import sys
import pandas as pd
from google.cloud import bigquery 
from google.oauth2 import service_account

key_path = "/home/alex/.creds/salex-sa.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

DATA_DIR = "../data"
PROJECT_NAME = "team-week2"

# **** TABLE SCHEMAS ****
TABLE_METADATA = {
   ## 'Raw' Data (just renaming columns)
   'chi_crimes': {
       'dataset_name':'chicago',
       'table_name': 'chi_crimes',
       'schema': [
           bigquery.SchemaField('crime_id', 'INTEGER', mode='REQUIRED'),
           bigquery.SchemaField('case_number', 'STRING', mode='REQUIRED'),
           bigquery.SchemaField('date_time', 'STRING', mode='NULLABLE'),
           bigquery.SchemaField('block', 'STRING', mode='NULLABLE'),
           bigquery.SchemaField('IUCR_code', 'STRING', mode='NULLABLE'),
           bigquery.SchemaField('primary_IUCR', 'STRING', mode='NULLABLE'),
           bigquery.SchemaField('secondary_IUCR', 'STRING', mode='NULLABLE'),
           bigquery.SchemaField('location_description', 'STRING', mode='NULLABLE'),
           bigquery.SchemaField('arrest', 'BOOL', mode='NULLABLE'),
           bigquery.SchemaField('domestic', 'BOOL', mode='NULLABLE'),
           bigquery.SchemaField('beat', 'INTEGER', mode='NULLABLE'),
           bigquery.SchemaField('district', 'INTEGER', mode='NULLABLE'),
           bigquery.SchemaField('ward', 'INTEGER', mode='NULLABLE'),
           bigquery.SchemaField('community_area', 'INTEGER', mode='NULLABLE'),
           bigquery.SchemaField('FBI_code', 'STRING', mode='NULLABLE'),
           bigquery.SchemaField('x_coordinate', 'FLOAT', mode='NULLABLE'),
           bigquery.SchemaField('y_coordinate', 'FLOAT', mode='NULLABLE'),
           bigquery.SchemaField('year', 'INTEGER', mode='NULLABLE'),
           bigquery.SchemaField('updated_on', 'STRING', mode='NULLABLE'),
           bigquery.SchemaField('lat', 'FLOAT', mode='NULLABLE'),
           bigquery.SchemaField('lon', 'FLOAT', mode='NULLABLE'), 
           bigquery.SchemaField('location', 'STRING', mode='NULLABLE'),
       ]
   }, 'den_crimes': {
        'dataset_name':'denver',
        'table_name':'den_crimes',
        'schema': [
            bigquery.SchemaField('incident_id', 'INTEGER', mode='REQUIRED'),
            bigquery.SchemaField('offense_id', 'INTEGER', mode='REQUIRED'),
            bigquery.SchemaField('offense_code', 'INTEGER', mode='NULLABLE'),
            bigquery.SchemaField('offense_code_ext', 'INTEGER', mode='NULLABLE'),
            bigquery.SchemaField('offense_type_id', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('offense_cat_id', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('first_occurrence', 'DATETIME', mode='NULLABLE'),
            bigquery.SchemaField('last_occurrence', 'DATETIME', mode='NULLABLE'),
            bigquery.SchemaField('reported_date', 'DATETIME', mode='NULLABLE'),
            bigquery.SchemaField('address', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('geo_x', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('geo_y', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('geo_lon', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('geo_lat', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('district_id', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('precinct_id', 'INTEGER', mode='NULLABLE'),
            bigquery.SchemaField('neighborhood_id', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('is_crime', 'BOOL', mode='NULLABLE'), 
            bigquery.SchemaField('is_traffic', 'BOOL', mode='NULLABLE'), 
            bigquery.SchemaField('victim_count', 'INTEGER', mode='NULLABLE')
          ] 
   }, 'off_codes':{
        'dataset_name':'denver',
        'table_name':'off_codes',
        'schema': [
            bigquery.SchemaField('object_id', 'INTEGER', mode='REQUIRED'),
            bigquery.SchemaField('offense_code', 'INTEGER', mode='REQUIRED'),
            bigquery.SchemaField('offense_code_ext', 'INTEGER', mode='NULLABLE'),
            bigquery.SchemaField('offense_type_id', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('offense_type_name', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('offense_cat_id', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('offense_cat_name', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('is_crime', 'BOOL', mode='NULLABLE'),
            bigquery.SchemaField('is_traffic', 'BOOL', mode='NULLABLE')

        ]
   }
}

# **** BIGQUERY CLIENT ****
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# **** CREATE TABLES  ****
for table_name in TABLE_METADATA:
  # Create dataset (if needed)
  dataset_name = TABLE_METADATA[table_name]['dataset_name']
  dataset_id = f"{PROJECT_NAME}.{dataset_name}"
  dataset = bigquery.Dataset(dataset_id)
  dataset.location = "US"
  dataset = client.create_dataset(dataset, exists_ok=True)
  # Create table
  full_table_id = f"{PROJECT_NAME}.{dataset_name}.{table_name}"
  schema = TABLE_METADATA[table_name]['schema']
  table = bigquery.Table(full_table_id, schema = schema)
  try:
    client.create_table(table)
    print(f"Created table: {table_name}")
    table_ref = client.get_table(table)
    for column in table_ref.schema:
      print(f"\t{column.name}\t{column.field_type}") 
    print("\n")
  except:
    print(f"Did not create table {table_name}. Already exists?")

Did not create table chi_crimes. Already exists?
Created table: den_crimes
	incident_id	INTEGER
	offense_id	INTEGER
	offense_code	INTEGER
	offense_code_ext	INTEGER
	offense_type_id	STRING
	offense_cat_id	STRING
	first_occurrence	DATETIME
	last_occurrence	DATETIME
	reported_date	DATETIME
	address	STRING
	geo_x	FLOAT
	geo_y	FLOAT
	geo_lon	FLOAT
	geo_lat	FLOAT
	district_id	STRING
	precinct_id	INTEGER
	neighborhood_id	STRING
	is_crime	BOOLEAN
	is_traffic	BOOLEAN
	victim_count	INTEGER


Did not create table off_codes. Already exists?


#### Load Chicago Crime Data

The Chicago Kaggle data contains yearly data tables from 2010 to 2022. We've limited ourselves to data since 2018 for this project. 

In [115]:
job_config = bigquery.LoadJobConfig(
        create_disposition="CREATE_IF_NEEDED",
        write_disposition="WRITE_APPEND",
        schema=TABLE_METADATA['chi_crimes']['schema']
    )

for year in range(18,23): 
  filepath = f"{DATA_DIR}/chicago/20{year}.csv"
  print(filepath)
  df = pd.read_csv(filepath)
  df.columns = [col.lower().replace(" ","_") for col in df.columns]
  df.rename({'id':'crime_id', 
          'date':'date_time',
          'iucr':'IUCR_code',
          'primary_type':'primary_IUCR',
          'description':'secondary_IUCR',
          'fbi_code':'FBI_code',
          'latitude':'lat',
          'longitude':'lon'
          }, axis=1, inplace=True)
  job = client.load_table_from_dataframe(df, destination=f'{PROJECT_NAME}.chicago.chi_crimes', job_config=job_config)
  job.result()

../data/chicago/2018.csv
../data/chicago/2019.csv
../data/chicago/2020.csv
../data/chicago/2021.csv
../data/chicago/2022.csv


#### Load Denver Crime Data

In [137]:
with open("../data/denver/crime.csv", "r", encoding="windows-1252") as file:
  den_crimes = pd.read_csv(file)

den_crimes.columns = [col.lower().replace("e_date","e") for col in den_crimes.columns]
den_crimes.rename({
  'offense_code_extension':'offense_code_ext',
  'offense_category_id':'offense_cat_id',
  'incident_address':'address'
}, axis=1, inplace=True)

den_crimes['first_occurrence'] = pd.to_datetime(den_crimes['first_occurrence'])
den_crimes['last_occurrence'] = pd.to_datetime(den_crimes['last_occurrence'])
den_crimes['reported_date'] = pd.to_datetime(den_crimes['reported_date'])


In [148]:
job_config = bigquery.LoadJobConfig(
        create_disposition="CREATE_IF_NEEDED",
        write_disposition="WRITE_TRUNCATE",
        schema=TABLE_METADATA['den_crimes']['schema']
    )
job = client.load_table_from_dataframe(
    den_crimes,
    destination=f"{PROJECT_NAME}.denver.den_crimes", 
    job_config=job_config
)
job.result()


LoadJob<project=team-week2, location=US, id=53a5896e-c7a1-41ed-a297-0e15ac465d5b>

In [166]:
## Offense Code Dimension Table
off_codes = pd.read_csv('../data/denver/offense_codes.csv')
off_codes.columns = [col.lower().replace("category","cat") for col in off_codes.columns]
off_codes.rename({
  'objectid':'object_id',
  'offense_code_extension':'offense_code_ext'
}, axis=1, inplace=True)

off_codes.dtypes

object_id             int64
offense_code          int64
offense_code_ext      int64
offense_type_id      object
offense_type_name    object
offense_cat_id       object
offense_cat_name     object
is_crime              int64
is_traffic            int64
dtype: object

In [167]:
job_config = bigquery.LoadJobConfig(
        create_disposition="CREATE_IF_NEEDED",
        write_disposition="WRITE_TRUNCATE",
        schema=TABLE_METADATA['off_codes']['schema']
    )
job = client.load_table_from_dataframe(
    off_codes,
    destination=f"{PROJECT_NAME}.denver.off_codes", 
    job_config=job_config
)
job.result()

LoadJob<project=team-week2, location=US, id=8d622a27-ca81-4483-bc0b-7ff63c56561c>

### Loading Chicago PD Map Geometries 

_[Source](https://data.cityofchicago.org/Public-Safety/Boundaries-Police-Beats-current-/aerh-rz74)_

Loading GeoJSON data to BigQuery is [somewhat finicky](https://stackoverflow.com/questions/24239056/load-geojson-in-bigquery). The easiest way I can think of doing this with Python is to load the geometries as strings in a dataframe and then use the `ST_GEOGFROMTEXT` function to convert them back to geometries once they're in BigQuery.

In [33]:
import geopandas as gpd

gdf = gpd.read_file("../data/chicago/Boundaries - Police Beats (current).geojson")
gdf

,beat,beat_num,district,sector,geometry
0,1,1713,17,1,"MULTIPOLYGON (((-87.70473 41.97577, -87.70472 ..."
1,0,3100,31,0,"MULTIPOLYGON (((-87.83365 41.97535, -87.83366 ..."
2,5,1651,16,5,"MULTIPOLYGON (((-87.90684 41.97656, -87.91070 ..."
3,1,1914,19,1,"MULTIPOLYGON (((-87.64492 41.96973, -87.64431 ..."
4,1,1915,19,1,"MULTIPOLYGON (((-87.63724 41.96599, -87.63644 ..."
...,...,...,...,...,...
272,1,0314,03,1,"MULTIPOLYGON (((-87.58879 41.78612, -87.58844 ..."
273,2,0825,08,2,"MULTIPOLYGON (((-87.67917 41.78656, -87.67912 ..."
274,1,0313,03,1,"MULTIPOLYGON (((-87.60854 41.78583, -87.60808 ..."
275,2,0823,08,2,"MULTIPOLYGON (((-87.69354 41.78458, -87.69354 ..."


In [37]:
gdf = gdf[['beat_num','district','geometry']]
df = pd.DataFrame(gdf) # Convert to regular dataframe before removing geometry typing
df['geometry'] = df['geometry'].astype(str)

In [39]:
job_config = bigquery.LoadJobConfig(
        create_disposition="CREATE_IF_NEEDED",
        write_disposition="WRITE_TRUNCATE",
        autodetect=True
    )
job = client.load_table_from_dataframe(
    df,
    destination="team-week2.chicago.geometries", 
    job_config=job_config
)
job.result()

LoadJob<project=team-week2, location=US, id=a3939764-d956-4cf9-8272-ebe8ee09d6eb>